<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/11_clustering_(example).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# （実行例）日本語の単語ベクトルをクラスタリング

* 使用するデータは下記の場所にあるもの

 * https://github.com/shiroyagicorp/japanese-word2vec-model-builder

* Dask-MLのspectral clusteringでクラスタリングして、意味の近い単語が同じクラスタに属しているかをチェックする。

In [1]:
!if [ ! -e latest-ja-word2vec-gensim-model.zip ]; then wget http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip ; fi

--2023-07-07 05:50:55--  http://public.shiroyagi.s3.amazonaws.com/latest-ja-word2vec-gensim-model.zip
Resolving public.shiroyagi.s3.amazonaws.com (public.shiroyagi.s3.amazonaws.com)... 52.219.150.161, 52.219.9.29, 52.219.150.33, ...
Connecting to public.shiroyagi.s3.amazonaws.com (public.shiroyagi.s3.amazonaws.com)|52.219.150.161|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132936751 (127M) [application/zip]
Saving to: ‘latest-ja-word2vec-gensim-model.zip’

latest-ja-word2vec- 100%[===================>] 126.78M  64.0MB/s    in 2.0s    

2023-07-07 05:50:57 (64.0 MB/s) - ‘latest-ja-word2vec-gensim-model.zip’ saved [132936751/132936751]



In [2]:
!unzip -u latest-ja-word2vec-gensim-model.zip
#!unzip -u /content/drive/MyDrive/data/latest-ja-word2vec-gensim-model.zip

Archive:  latest-ja-word2vec-gensim-model.zip
  inflating: word2vec.gensim.model   
  inflating: word2vec.gensim.model.syn1neg.npy  
  inflating: word2vec.gensim.model.wv.syn0.npy  


In [3]:
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)

In [4]:
wv = model.wv

In [6]:
words = []
vectors = []
for word in wv.key_to_index.keys():
  words.append(word)
  vectors.append(wv.get_vector(word))

In [8]:
import numpy as np

words = np.array(words)
vectors = np.array(vectors)

## Dask-MLのspectral clustering

* Dask-MLについてはWebサイトを参照
 * https://ml.dask.org/

* scalableなspectral clustering（ただし近似が入る）が実装されているので、使ってみる。
 * https://ml.dask.org/clustering.html
* sklearnにもspectral clusteringはある。
 * https://scikit-learn.org/stable/modules/generated/sklearn.cluster.SpectralClustering.html
 * こちらは近似が入らない代わりに、遅い。
* 参考
 * https://dask-ml-benchmarks.readthedocs.io/en/latest/auto_examples/plot_spectral_clustering.html

In [9]:
!pip install dask_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.7/148.7 kB 3.8 MB/s eta 0:00:00


* 単語ベクトルのデータを取得し直す



In [10]:
import numpy as np
from gensim.models.word2vec import Word2Vec

model_path = 'word2vec.gensim.model'
model = Word2Vec.load(model_path)
wv = model.wv
words = list()
vectors = list()
for word in wv.key_to_index.keys():
  words.append(word)
  vectors.append(wv.get_vector(word))
words = np.array(words)
vectors = np.array(vectors)

In [11]:
import dask_ml.cluster

n_clusters = 100
clusterer = dask_ml.cluster.SpectralClustering(
    n_clusters=n_clusters,
    persist_embedding=True,
    kmeans_params={"init":"k-means++"},
    random_state=123,
    )

In [12]:
%%time
clusterer.fit(vectors)

CPU times: user 3min 1s, sys: 10.6 s, total: 3min 11s
Wall time: 2min 35s


SpectralClustering(kmeans_params={'init': 'k-means++'}, n_clusters=100,
                   persist_embedding=True, random_state=123)

In [13]:
labels = clusterer.labels_.compute()

In [14]:
np.save('dask_sc_labels', labels)

In [15]:
labels

array([ 5, 30, 14, ...,  2,  1, 57], dtype=int32)

* クラスタのサイズ順にクラスタのインデックス（0~99）をソートする。

In [16]:
unique, counts = np.unique(labels, return_counts=True)
size_dict = dict(zip(unique, counts))
sorted_clusters = [k for k, v in sorted(size_dict.items(), key=lambda item: item[1], reverse=True)]

### クラスタのサイズを調べる

In [17]:
counts[sorted_clusters]

array([8204, 7208, 5701, 4965, 4895, 4778, 4718, 4713, 4708, 4659, 4646,
       4579, 4481, 4451, 4347, 4324, 4287, 4270, 4184, 4143, 4120, 4097,
       4074, 4017, 3976, 3860, 3847, 3783, 3730, 3729, 3690, 3662, 3592,
       3569, 3561, 3457, 3450, 3348, 3344, 3339, 3329, 3281, 3258, 3257,
       3250, 3232, 3231, 3226, 3221, 3210, 3162, 3125, 3118, 3094, 3086,
       3079, 3059, 3055, 3053, 3040, 3012, 2976, 2972, 2935, 2864, 2852,
       2848, 2829, 2828, 2781, 2781, 2770, 2741, 2727, 2680, 2679, 2671,
       2643, 2635, 2627, 2608, 2599, 2587, 2573, 2571, 2560, 2531, 2487,
       2459, 2349, 2348, 2307, 2189, 2143, 2097, 2054, 1939, 1369, 1321,
        662])

### サイズが最大のクラスタを調べる

In [18]:
' '.join(words[labels == sorted_clusters[0]])

'松平正容 ユニットコム 福山中学校 広島県立福山誠之館高等学校 公立大学法人首都大学東京 小田喜 山本精一 マインド・ゲーム ポルデノーネ無声映画祭 京都大学大学院法学研究科 東京都立墨田川高等学校 六神合体ゴッドマーズ サンダーマスク 飯島敏宏 スーパーロボット レッドバロン ワンサくん プラレス3四郎 機甲猟兵メロウリンク 忍者戦隊カクレンジャー 超獣戦隊ライブマン 無限増殖 魔界村 狩野常信 especially フランケンシュタインの逆襲 浮草 6120 シークレット・ドクトリン ユーシーカード株式会社 吉助 大阪府大阪市平野区 ~、 田中学 ガンパウダー プリンスオブウェールズ 宮城県警察部 マエストラーレ 東海艦隊 南海艦隊 シレ 昭洋 世界フィギュアスケート国別対抗戦 おいでよ どうぶつの森 33打点 熊姫 ザ・イロモネア 小林麻耶 世界の果てまで セブン・アークス 9370 3080 Tatiana ラジオどっとあい 白峯寺 狩野安信 武蔵 MUSASHI ヒューマ 新地駅 鬼武者 秋元玲奈 RSIP ACD 公証人法 尽誠学園 力弥 オオ とっくり ここな 片岡我童 大岸 さえぎっ むざむざ 久松静児 黒部の太陽 海と毒薬 Outrage 1246 のろう パワプロクンポケット ハンゲ スタジオジャイアンツ 王立宇宙軍 オネアミスの翼 スターデジオ フリクリ 福岡県立小倉高等学校 福岡4区 滋賀県立彦根東高等学校 三菱重工業株式会社 エンガノ 超時空世紀オーガス 超時空騎団サザンクロス 機甲創世記モスピーダ 大鉄人17 超時空要塞マクロスII -LOVERS AGAIN- たかべ ルイジ・コメンチーニ 一興 予防外交 沖風 梶岡定道 丸茂 日本リーバ 機動戦士ガンダム 戦場の絆 サラマンドラ "|-|||||||| Stéphane Florent 育英高等学校 雲雀丘 3720 1595 朝夜 十日市場 松本電鉄バス テアトロン 砂の女 定州市 ウルトラジャンプ 月刊電撃コミックガオ! 別冊ヤングマガジン 特警ウインスペクター 地獄先生ぬ〜べ〜 Grosso 五星戦隊ダイレンジャー 太陽の王子 ホルスの大冒険 子鹿物語 デュアルヘッド hydroxide 利章 川崎市市民ミュージアム gsi 1583 Rémy ベール湖 メトロポール・ド・リヨ

### サイズが最小のクラスタを調べる

In [19]:
' '.join(words[labels == sorted_clusters[99]])

'求めよ 連れ戻そ だろ しよ ご 戻ろ 盗も あろ 受け取ろ 抑えよ 登ろ けんど 戻そ 取り持と つこ させよ 押し通そ せよ 示そ 集めよ こんど 出そ 討と 高めよ 行なお 語ろ 納めよ 言お 渡そ 及ぼ でしょ 立と 抜こ 斬ろ やろ 付けよ 見よ 殺そ 収めよ 立ち去ろ 引き返そ 慰めよ 切ろ いえよ 返そ もらお ましょ 帰ろ 見極めよ 殴ろ 探ろ してやろ くれよ かかろ まいと さそ 奪お あげよ 聞こ 刺そ 買お 捕らえよ 送ろ 贈ろ 戦お ちろ 備えよ ぶろ 迎えよ ほしょ えんど 決めよ 伝えよ 知らせよ 遊ぼ かいきょ 受けよ かんす 出よ いこ おこ 立て直そ かわそ 渡ろ たろ 保と かけよ 与えよ 役立てよ 行お 得よ 逃げ出そ 助けよ 近づこ 合わせよ ていりゅうじょ 洗お みよ 始まろ 図ろ 彷徨 届けよ 減らそ 辞めよ なろ 立てよ られよ 起こそ 封じよ 結ぼ 調べよ 救お ごまかそ 避けよ 抱きしめよ 防ご 歌お 守ろ 上げよ 導こ 述べよ 割ろ 打と 死の いよ 見せよ よかろ 取り戻そ 売り出そ 変えよ 目指そ 逃げよ 乗ろ 言えよ 妨げよ 知ろ 乗り切ろ 作り上げよ 育てよ 入れよ 認めよ 及ぼそ 加えよ 作ろ しじょ 乗り込も 留めよ 尽くそ 働こ 続けよ 組み込も 潰そ 広げよ 挑も 探そ げんこ 進も 座ろ 掴も ぎゅ 見出そ つけよ 書こ 倒そ 見直そ 開こ 追い払お 乗っ取ろ 用いよ 譲ろ 行こ 追い出そ れよ 説こ 絶する 置こ づけよ 笑お 盛り上げよ 食お やめよ じゃろ できよ 狙お 止めよ 盛り込も ぶしゅ 葬ろ 通り過ぎよ 取ろ 始めよ 捨てよ 応えよ 逃れよ 補お 握ろ 進めよ 強めよ 当たろ 起きよ 壊そ 描こ 採ろ 貫こ 持と 使お 極めよ 抜け出そ 果して 消そ 会お きんじろ 出かけよ 召し出そ ぎじょ とうじょ 貸そ 触ろ げつよ 仕立てよ 覚えよ 明かそ 済ませよ 稼ご 去ろ 貰お 捕まえよ 滅ぼそ 逃がそ 移そ 確かめよ 消えよ つくろ むそ 退けよ 就こ 封じ込めよ 暴こ 躍起 応じよ 仕えよ 定めよ 支えよ 攻めよ 考えよ 話そ 攻め込も 引こ 掛けよ 飛び込も 動かそ 隠そ 伸ばそ 持ち出そ はかろ もたらそ 売り込も 裂こ 引き出そ 挙げよ 広めよ 取り上げよ 深めよ はも

### サイズが中間的なクラスタを調べる

In [20]:
' '.join(words[labels == sorted_clusters[49]])

'】 \u3000 文学部 自治医科大学 学 麹町中学校 和光大学 首都大学東京 修士課程 関西大学 社会学部 台湾大学 法律学科 専攻 単位取得退学 京都大学 研究会 日本の教育 府立一中 東京都立日比谷高等学校 高等工業学校 工業大学 基礎科学 紫綬褒章 東京都立新宿高等学校 日本工芸会 日本文芸家協会 日本ペンクラブ 日本脚本家連盟 京都嵯峨芸術大学 客員教授 東京都出身 慶應義塾大学 国文 卒 慶大 名誉教授 東京大学 理学部 数物 研究員 東京神学大学 学長 福岡県出身 情報学 大阪大学 国立新美術館 上智大学 文科 教養学部 同大 東京大学東洋文化研究所 立教大学 助教授 人間科学部 阪大 人間科学 東京大学先端科学技術研究センター 政策研究大学院大学 法政大学 特任教授 早稲田大学 研究所 学会 青山学院大学 県立 女子高等師範学校 広島大学 教育学部 男子寮 香川県出身 公益財団法人 文化振興 芸術文化 神奈川県横浜市 工学部 応用化学 福岡大学 研 歴任 高等師範学校 上海大学 中国人民政治協商会議全国委員会 宮城師範学校 宮城県図書館 日本体育大学 体育学部 功労賞 文化功労者 日本芸術院 東亜大学 京都府京都市 府立 文学研究科 芸術選奨新人賞 恩賜賞 プレップスクール 東大 岡山県出身 理工学部 フランス文学者 身体運動 行政学者 日本学士院 東京都生まれ 法学部 国際基督教大学 研究科 東京市政調査会 21世紀臨調 リトルシニア 大阪府出身 文化財研究所 財政学 経営学 東京藝術大学 日本現代音楽協会 総合科学 体育学 全日本スキー連盟 BFA 一橋大学 アミューズメントメディア総合学院 ロボット競技 照明デザイナー 常磐大学 地学 NHK放送文化研究所 大阪外国語大学 ）< 中央民族大学 ティーチングアシスタント 電気工学 サイモンフレーザー 西オーストラリア大学 内科学 明治大学 商学部 医員 帝京 信州大学 文理学部 芸術選奨 愛知県名古屋市 全日本軟式野球連盟 審判学校 国士舘大学 東海大学 筑波大学 工学 関西学院大学 福井県立藤島高等学校 経済学部 農林水産 経営学部 修道中学校 前橋中学校 陸軍歩兵学校 医学博士 九州大学 学芸大学 健康科学部 滋賀県出身 東京都立大学附属高等学校 山梨県出身 歴史教育者協議会 日本政治史 政

* 全てのクラスタについて、**ランダムな50語**を表示

In [21]:
for i in range(len(sorted_clusters)):
  members = words[labels == sorted_clusters[i]]
  np.random.shuffle(members)
  print(' '.join(members[:50]))
  print('-'*80)

餓狼 MARK OF THE WOLVES Grosso 閃乱カグラ 日本李登輝友の会 高口 Xfire 月刊COMICリュウ 秋山瑞人 2246 嘉川 Syria ベネズエラ・ボリバル共和国 キャサリン・スウィンフォード アルバネーゼ ロロナ SAINT まむしの兄弟 ディジット ペチョラ デイリーイン 株式会社東芝 山口県立豊浦高等学校 終末から 三陽 ドルマバフチェ 椎名慶治 新地駅 三国伝 試し切り 龍三郎 サーベラス・キャピタル・マネジメント 公立学校共済組合 ガジェットリンク 2275 松平頼純 瓊浦 徳島地方裁判所 78打点 サモトラキ 新・のび太と鉄人兵団 ダブルオーセブン ごちそうさま 131度 アタック! 脳動静脈奇形 ∑ バイオレンスジャック シーアンドティー Silvia 虎鉄
--------------------------------------------------------------------------------
交情 センサネットワーク PAG パシフィックグランプリ OKICA カリーフ トランスアジア航空 骨盤骨折 永田川 外道乙女隊 漠然 オペラ・オーストラリア バロウ decyclizing クロゾン 神丸 芦屋学園 ノーデンス ベヴィン ニック・キリオス 田知本愛 揚幕 Googleビデオ ジャパンボウル カカオトーク 薬害エイズ 八股文 スウィート・ホーム TAGRO JBOD ハローグッバイ 勝英 マスターガンダム テレビくん 四村 びさい ターヒル ゲンリフ・ネイガウス ニューブリード 島峰 フリッツ・ザ・キャット 五匁銀 トリツ ヘ音記号 ダイヤモンドは永遠に プベル プリンス・オブ・ペルシャ 雒城 アクロイヤー 孫呉
--------------------------------------------------------------------------------
99分 佐藤アツヒロ ブギーマン Riryka アップルシード シーメール アクエリアンエイジ ホパロング・キャシディ VANGUARD I miss you プロレスリング華☆激 魔法騎士レイアース オールナイトロング ライアーソフト 冬物語 東京少年 61分 MERY ララベル 真・三國無双 EGO-WRAPPIN' 鉄